In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [18]:
vdf = pd.read_csv('/Users/willmayer/Desktop/DS3001/project/DS3001-Final-Project/data/voting_VA.csv', low_memory=False)

path = '/Users/willmayer/Desktop/DS3001/project/DS3001-Final-Project/data/county_data/utf8_justVA'
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]

dfs = {}
for file in csv_files:
    file_path = os.path.join(path, file)
    dfs[file] = pd.read_csv(file_path)

x = dfs.keys()
print(x)

# df = pd.merge(vdf, cdf, left_on='county_fips', right_on='FIPS')
# df = gdf.merge(cdf,left_on='FIPS_left',right_on='FIPS')  # merge county population and adjacency data into df
# del gdf, cdf

dict_keys(['0002_ds250_20205_county_E.csv', '0002_ds176_20105_county_M.csv', '0002_ds206_20145_county_E.csv', '0002_ds225_20165_county_M.csv', '0002_ds191_20125_county_M.csv', '0002_ds177_20105_county_E.csv', '0002_ds249_20205_county_M.csv', '0002_ds239_20185_county_E.csv', '0002_ds249_20205_county_E.csv', '0002_ds239_20185_county_M.csv', '0002_ds250_20205_county_M.csv', '0002_ts_nominal_county.csv', '0002_ds206_20145_county_M.csv', '0002_ds176_20105_county_E.csv', '0002_ds225_20165_county_E.csv', '0002_ds177_20105_county_M.csv', '0002_ds191_20125_county_E.csv'])


In [19]:
dfs['0002_ds250_20205_county_E.csv']['YEAR'] = 2020
dfs['0002_ds176_20105_county_M.csv']['YEAR'] = 2008
dfs['0002_ds206_20145_county_E.csv']['YEAR'] = 2012
dfs['0002_ds225_20165_county_M.csv']['YEAR'] = 2016
dfs['0002_ds191_20125_county_M.csv']['YEAR'] = 2012
dfs['0002_ds177_20105_county_E.csv']['YEAR'] = 2008
dfs['0002_ds249_20205_county_M.csv']['YEAR'] = 2020
dfs['0002_ds239_20185_county_E.csv']['YEAR'] = 2016
dfs['0002_ds249_20205_county_E.csv']['YEAR'] = 2020
dfs['0002_ds239_20185_county_M.csv']['YEAR'] = 2016
dfs['0002_ds250_20205_county_M.csv']['YEAR'] = 2020
dfs['0002_ds206_20145_county_M.csv']['YEAR'] = 2012
dfs['0002_ds176_20105_county_E.csv']['YEAR'] = 2008
dfs['0002_ds225_20165_county_E.csv']['YEAR'] = 2016
dfs['0002_ds177_20105_county_M.csv']['YEAR'] = 2008
dfs['0002_ds191_20125_county_E.csv']['YEAR'] = 2012
del dfs['0002_ts_nominal_county.csv']

In [20]:
stacked_r = pd.concat(dfs.values(),ignore_index=True)
stacked_r.rename(columns={'YEAR':'year','COUNTYA':'county_fips'}, inplace=True)
vdf['county_fips'] = vdf['county_fips'] - 51000
vdf = pd.merge(vdf, stacked_r, on=['county_fips', 'year'], how='inner')

In [21]:
vdf

,Unnamed: 0,year,state,state_po,county_name,county_fips,office,candidate,party,candidatevotes,...,QXSE007,QX6E001,QX7E001,QX7E002,QX7E003,QX8E001,QX8E002,QX8E003,QZTE001,QZ6E001
0,30189,2008,VIRGINIA,VA,ACCOMACK,1,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30189,2008,VIRGINIA,VA,ACCOMACK,1,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30189,2008,VIRGINIA,VA,ACCOMACK,1,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30189,2008,VIRGINIA,VA,ACCOMACK,1,US PRESIDENT,BARACK OBAMA,DEMOCRAT,7607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30190,2008,VIRGINIA,VA,ACCOMACK,1,US PRESIDENT,JOHN MCCAIN,REPUBLICAN,7833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11169,71969,2020,VIRGINIA,VA,WINCHESTER CITY,840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,2370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11170,71970,2020,VIRGINIA,VA,WINCHESTER CITY,840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11171,71970,2020,VIRGINIA,VA,WINCHESTER CITY,840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11172,71970,2020,VIRGINIA,VA,WINCHESTER CITY,840,US PRESIDENT,DONALD J TRUMP,REPUBLICAN,26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
